In [49]:
from soynlp.word import WordExtractor
import pandas as pd
import re
from itertools import chain

In [4]:
df = pd.read_pickle('df_dataset.pkl')

In [9]:
corpus = df.text.tolist()

,text,x,y
0,또..주문 합니다 잇몸에 자극이 없어서 좋아욤,"[주문, 하, 잇, 몸, 자극, 없, 좋, 욤]",1
1,흡수력은 좋은고 같구요 다른 비타민씨제품이 요즘 약간 속당김이 느껴지는거 같아 바꿔...,"[흡수, 력, 좋, 같, 비타민, 씨, 제품, 속, 당기, 느끼, 지, 것, 같, ...",1
2,이너로 입기에도 충전재가 약하네요;;\n 조끼 빼고 겉옷 위에 털조끼 입고 외출 했...,"[이너, 입, 충전, 재, 약하, 조끼, 빼, 겉, 옷, 위, 털, 조끼, 입, 하...",0
3,딸아이 코에 해봤는데 잘 안빠지네요~ㅠㅠ\n목욕후 모공이 열렸을때 다시 해보려구요.,"[딸, 아이, 코, 하, 보, 빠지, ㅠㅠ, 목욕, 후, 모공, 열리, 때, 하, 보]",1
4,빠른 배송 감사합니다. 아이가 너무 마음에 들어 합니다.,"[빠르, 배송, 감사, 아이, 마음, 듣, 하]",1


In [13]:
def cleansing_text(text):
    text = re.sub(pattern='\n|ㆍ|\r|\.|,|;|:|<|>|-|_|=|\+|/|\?|!|@|\(|\)|\*|&|！|\^|#|%|\$|~|？|\[|\]|\"|[0-9]', repl=" ", string=text)
    text = re.sub(pattern='( )+', repl=' ', string=text)
    text = re.sub(pattern='(ㅎ)+', repl='ㅎ', string=text)
    text = re.sub(pattern='(ㅋ)+', repl='ㅋ', string=text)
    text = re.sub(pattern='(ㅠ|ㅜ)+', repl='ㅠ', string=text)
    return text

In [14]:
corpus = [cleansing_text(text) for text in corpus]

In [15]:
corpus[:10]

['또 주문 합니다 잇몸에 자극이 없어서 좋아욤',
 '흡수력은 좋은고 같구요 다른 비타민씨제품이 요즘 약간 속당김이 느껴지는거 같아 바꿔볼까 고민중입니다',
 '이너로 입기에도 충전재가 약하네요 조끼 빼고 겉옷 위에 털조끼 입고 외출 했는데 바람 숭숭 들어 오네요 추워요 보온성 아쉬워요 ',
 '딸아이 코에 해봤는데 잘 안빠지네요 ㅠ 목욕후 모공이 열렸을때 다시 해보려구요 ',
 '빠른 배송 감사합니다 아이가 너무 마음에 들어 합니다 ',
 '정말 내 돈주고 구입해서 성질버린다라는 표현생각나는 제품 털고 찍찍이로 굴리고 청소기로 빨아내고 또 찍찍이로 찍고를 했는데 어제 첫눈와서 잠깐 두르고 나갔다가 헉 이런제품 팔지 맙시다\U0001f928',
 '좋아요게소그는데에센스는윤조가제일인듯',
 '쫀쫀한건 좋은데 입고벗기가 기술이 필요할듯 싸이즈 교환으로 교환비까지 ㅠ',
 '역시 로보네요 타방송 무스탕 입어보고 반품만 두번 로보를 입어보고 이거다 싶어 기쁜 마음으로 입습니다',
 '가볍습니다 발볼이 넓은 사람은 불편할듯']

In [15]:
word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

In [17]:
word_extractor.train(corpus)

training ... (0 in 133606 sents) use memory 0.209 Gb

training ... (100000 in 133606 sents) use memory 0.604 Gb

training was done. used memory 0.630 Gb


In [18]:
words = word_extractor.extract()

all cohesion probabilities was computed. # words = 6315


all branching entropies was computed # words = 105001


all accessor variety was computed # words = 105001


In [20]:
words_v2 = {k: v for k, v in words.items() if len(k) > 1}

In [37]:
words_v3 = list()
for k, v in words_v2.items():
    temp = dict()
    cohesion = v.cohesion_forward
    branching_entropy = v.left_branching_entropy
    left_freq = v.leftside_frequency
    right_freq = v.rightside_frequency
    score = cohesion * branching_entropy
    
    temp['word'] = k
    temp['cohesion'] = cohesion
    temp['branching_entropy'] = branching_entropy
    temp['left_freq'] = left_freq
    temp['right_freq'] = right_freq
    temp['score'] = cohesion * branching_entropy
    words_v3.append(temp)

In [38]:
df_word = pd.DataFrame(words_v3)

In [39]:
words_v2

{'껍지': Scores(cohesion_forward=0, cohesion_backward=0.008636092912447886, left_branching_entropy=0, right_branching_entropy=0.08790889723203596, left_accessor_variety=0, right_accessor_variety=2, leftside_frequency=0, rightside_frequency=174),
 '바른': Scores(cohesion_forward=0.017442348008385743, cohesion_backward=0, left_branching_entropy=3.003190021052345, right_branching_entropy=1.9534735649495605, left_accessor_variety=28, right_accessor_variety=10, leftside_frequency=208, rightside_frequency=0),
 '라운': Scores(cohesion_forward=0, cohesion_backward=0.06898817345597898, left_branching_entropy=2.1972872109880957, right_branching_entropy=3.278410782359412, left_accessor_variety=12, right_accessor_variety=39, leftside_frequency=0, rightside_frequency=210),
 '작게': Scores(cohesion_forward=0.1354071363220494, cohesion_backward=0, left_branching_entropy=2.622757069648559, right_branching_entropy=2.0214699331780674, left_accessor_variety=37, right_accessor_variety=28, leftside_frequency=888, 

In [44]:
df_word.loc[df_word.score > 2].sample(20).sort_values(by=['score'], ascending=False)

,branching_entropy,cohesion,left_freq,right_freq,score,word
1427,3.488127,0.988906,624,0,3.449431,앰플
508,3.572027,0.781573,3249,0,2.791801,머리
328,2.865654,0.973684,185,0,2.790242,칙칙
1124,2.783315,0.968401,521,0,2.695367,톡톡
3947,2.631646,0.982424,277,0,2.585392,굉장히
2065,2.791745,0.922297,273,0,2.574819,칫솔
1897,3.585904,0.712156,2197,0,2.553722,얼굴
1332,3.131422,0.790914,6302,0,2.476685,생각
2071,2.661424,0.918951,771,0,2.445719,판매
85,2.634325,0.927536,256,0,2.443432,며칠


In [47]:
# len(df_word.loc[df_word.score > 1])
len(df_word)

4912

In [68]:
term_list = df.x.tolist()
term_list = [term for term in chain(*term_list)]
term_list = list(set(term_list))

In [54]:
candi_list = df_word.loc[df_word.score > 1].word.tolist()

In [69]:
candi_set = set(candi_list)
term_set = set(term_list)

In [70]:
candi_set_v2 = candi_set - term_set

In [73]:
len(candi_set_v2)

457

In [75]:
df_candi = pd.DataFrame(list(candi_set_v2))

In [78]:
df_candi.columns = ['word']
df_candi = df_candi.sort_values(by=['word'])

In [79]:
df_candi.to_csv('./token_candidate.csv', index=False, header=False)